# TT1 - MDM UBA - 2025

**Tariff classification using NLP**

By Santiago Tedoldi

## Goods description EDA

In [ ]:
# Dependencies
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from transformers import DistilBertTokenizerFast, DistilBertModel
import torch
from torch.utils.data import Dataset, DataLoader


### Raw dataset

In [2]:
colspecs = [(0, 6), (6, None)]
data_type = {'HS06': str}
df = pd.read_fwf('data/raw_data_HScodes_desc.txt',
                 colspecs=colspecs, header=None,
                 names=['HS06', 'GOODS_DESCRIPTION'],
                 dtype=data_type)

### Quick EDA

null and duplicated samples

dropping duplicates

analyzing tops and bottoms regarding frequencies

In [3]:
# Quick EDA
print("=== Quick EDA ===")

# Add HS02 (chapter) and HS04 (heading)
df['HS04'] = df['HS06'].str[:4]
df['HS02'] = df['HS06'].str[:2]

print("Nulls per column:")
print(df.isnull().sum(), "\n")

print("Duplicate rows:", df.duplicated().sum(), "\n")

# Function to build and display freq tables
def freq_table(col, name):
    vc      = df[col].value_counts().rename('count')
    rel     = df[col].value_counts(normalize=True).rename('rel_freq')
    cum     = rel.cumsum().rename('cum_freq')
    summary = pd.concat([vc, rel, cum], axis=1)
    summary['rel_freq'] = (summary['rel_freq'] * 100).round(2).astype(str) + '%'
    summary['cum_freq'] = (summary['cum_freq'] * 100).round(2).astype(str) + '%'

    print(f"## Samples per {name} ({col})\n")
    print("### Top 10")
    print(summary.head(10).to_markdown(), "\n")
    print("### Bottom 10")
    print(summary.tail(10).to_markdown(), "\n")

# Dropping duplicates
df.drop_duplicates(inplace=True)

# Chapter-level (HS02)
freq_table('HS02', 'chapter')

# Heading-level (HS04)
freq_table('HS04', 'heading')

# Subheading-level (HS06)
freq_table('HS06', 'subheading')

=== Quick EDA ===
Nulls per column:
HS06                 0
GOODS_DESCRIPTION    0
HS04                 0
HS02                 0
dtype: int64 

Duplicate rows: 232220 

## Samples per chapter (HS02)

### Top 10
|   HS02 |   count | rel_freq   | cum_freq   |
|-------:|--------:|:-----------|:-----------|
|     84 |   54901 | 20.5%      | 20.5%      |
|     85 |   33571 | 12.54%     | 33.04%     |
|     87 |   28476 | 10.63%     | 43.67%     |
|     73 |   16173 | 6.04%      | 49.71%     |
|     39 |   12218 | 4.56%      | 54.28%     |
|     90 |   11611 | 4.34%      | 58.61%     |
|     82 |    7972 | 2.98%      | 61.59%     |
|     94 |    7921 | 2.96%      | 64.55%     |
|     40 |    7526 | 2.81%      | 67.36%     |
|     83 |    4285 | 1.6%       | 68.96%     | 

### Bottom 10
|   HS02 |   count | rel_freq   | cum_freq   |
|-------:|--------:|:-----------|:-----------|
|     41 |      22 | 0.01%      | 99.96%     |
|     81 |      19 | 0.01%      | 99.97%     |
|     45 |      19 | 0

In [4]:
df

,HS06,GOODS_DESCRIPTION,HS04,HS02
0,271019,BRAKE FLUID DOT 4 50X200ML,2710,27
1,847710,PLASTIC INJECTION MOULD MODEL 21A 110G DSM1010...,8477,84
2,844399,LCD ASSEMBLY,8443,84
3,848280,BEARING 22238 KCAW33C3 BRAND MCB,8482,84
4,630900,USED HANDBAGS AND WALLETS,6309,63
...,...,...,...,...
499959,854239,PCB OPTIONAL ADD. KROPT V4.0 (NEW OUT PUT CARD),8542,85
499961,842091,CYLINDER (SDA80*10F003000001A),8420,84
499970,830249,BEOTIC DEVICE,8302,83
499981,901180,COMPOUND BINOCULAR MICROSCOPE,9011,90


Merging with HS06 nomenclature

In [5]:
# # Reading eng - kor description file 
# df_hs_eng_kor = pd.read_pickle('data/hs6_commentary.pickle')

# df_hs_eng_kor

In [6]:
# # Make sure your hs column is a string
# df_hs_eng_kor['hs'] = df_hs_eng_kor['hs'].astype(str)

# # uild a unique map hs → English description
# #    If there are duplicate (hs,eng) rows, drop duplicates;
# #    if there are truly conflicting descriptions for the same code,
# #    this will keep the *last* one in the DataFrame.
# desc_map = (
#     df_hs_eng_kor[['hs','eng']]
#     .drop_duplicates(subset=['hs'], keep='last')
#     .set_index('hs')['eng']
#     .to_dict()
# )

# # Find every true 6-digit code in your data…
# six_digits   = df_hs_eng_kor[df_hs_eng_kor['hs'].str.len() == 6]['hs'].unique().tolist()
# # and every 5-digit code (we’ll treat these as 6-digit by adding a “0”)
# five_digits  = df_hs_eng_kor[df_hs_eng_kor['hs'].str.len() == 5]['hs'].unique().tolist()
# padded_six   = [c + '0' for c in five_digits]
# # Combine into the full list of target 6-digit codes
# all_six = sorted(set(six_digits + padded_six))

# # Build a new DataFrame to hold them
# out = pd.DataFrame({'hs06': all_six})

# # For each code, pull out its 4-digit and 5-digit “parents”:
# out['lvl4'] = out['hs06'].str[:4]   # e.g. "0101"
# out['lvl5'] = out['hs06'].str[:5]   # e.g. "01012"
# out['lvl6'] = out['hs06']           # e.g. "010121" or "010130"

# # Look up the English text for each level (will be NaN if missing)
# out['eng4'] = out['lvl4'].map(desc_map)
# out['eng5'] = out['lvl5'].map(desc_map)
# out['eng6'] = out['lvl6'].map(desc_map)

# # Concatenate whatever levels you found, in order
# def make_full(row):
#     return ' && '.join(
#         filt for filt in (row['eng4'], row['eng5'], row['eng6'])
#         if pd.notnull(filt)
#     )

# out['full_eng'] = out.apply(make_full, axis=1)

# # Select just the two columns you need
# print(out[['hs06', 'full_eng']].to_markdown())

# # Saving into hs06_full_eng
# out[['hs06', 'full_eng']].to_csv('data/hs06_full_eng.csv')


In [7]:
df_hs06 = pd.read_csv('data/hs06_full_eng.csv', index_col='hs06', 
                      dtype={'hs06': str, 'full_eng': str},
                      usecols=['hs06', 'full_eng'])

In [8]:
# top 5 rows in HS06 nomemclature
print(df_hs06.head(5).to_markdown(), "\n")

# bottom 5 rows in HS06 nomemclature
print(df_hs06.tail(5).to_markdown(), "\n")

|   hs06 | full_eng                                                                              |
|-------:|:--------------------------------------------------------------------------------------|
| 010120 | Live horses, asses, mules and hinnies. && - Horses :                                  |
| 010121 | Live horses, asses, mules and hinnies. && - Horses : && -- Pure-bred breeding animals |
| 010129 | Live horses, asses, mules and hinnies. && - Horses : && -- Other                      |
| 010130 | Live horses, asses, mules and hinnies. && - Asses                                     |
| 010190 | Live horses, asses, mules and hinnies. && - Other                                     | 

|   hs06 | full_eng                                                                                                                                                                                                                                             |
|-------:|:------------------------------------

In [9]:
df = pd.merge(df, df_hs06,how='left', left_on='HS06', right_on='hs06')

In [10]:
print("Nulls per column:")
print(df.isnull().sum()/len(df), "\n")

Nulls per column:
HS06                 0.000000
GOODS_DESCRIPTION    0.000000
HS04                 0.000000
HS02                 0.000000
full_eng             0.045381
dtype: float64 



There are 4.5 % of goods with no HS full_eng available

They may are not updated codes

### Deep EDA

aggregate text statistics by HS level

Utils

In [11]:
# Utils f
def hs_frequencies_process(df, hs_codification = []):
    for hs_codi in hs_codification:

        df = df.merge(df[hs_codi].value_counts(), left_on=hs_codi, right_index=True)
        df.rename(columns={'count':f'{hs_codi}_samples'}, inplace=True)

    return df

def description_length(df, description_cols = []):
    for col in description_cols:

        df[f'{col}_len_words'] = df[col].apply(lambda x: len(x.split()))
        df[f'{col}_len_chars'] = df[col].apply(lambda x: len(x))

    return df

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

vocab = tokenizer.get_vocab()

def subtokenization_indicator(description):
    words = description.lower().split()
    tokens = tokenizer.tokenize(description)
    return len(tokens)/len(words)

In [12]:
df.head()

,HS06,GOODS_DESCRIPTION,HS04,HS02,full_eng
0,271019,BRAKE FLUID DOT 4 50X200ML,2710,27,Petroleum oils and oils obtained from bitumino...
1,847710,PLASTIC INJECTION MOULD MODEL 21A 110G DSM1010...,8477,84,Machinery for working rubber or plastics or fo...
2,844399,LCD ASSEMBLY,8443,84,Printing machinery used for printing by means ...
3,848280,BEARING 22238 KCAW33C3 BRAND MCB,8482,84,"Ball or roller bearings. && - Other, including..."
4,630900,USED HANDBAGS AND WALLETS,6309,63,NaN


HS frequencies

In [13]:
# done below with group_by for stats
# df = hs_frequencies_process(df, hs_codification=["HS06", "HS04", "HS02"])

Description lenghts

In [14]:
df = description_length(df, description_cols=['GOODS_DESCRIPTION'])

Subtokenization indicator

In [15]:
df['subtokenization_indicator'] = df['GOODS_DESCRIPTION'].apply(subtokenization_indicator)

In [16]:
df.sort_index(inplace=True)

In [17]:
df.columns

Index(['HS06', 'GOODS_DESCRIPTION', 'HS04', 'HS02', 'full_eng',
       'GOODS_DESCRIPTION_len_words', 'GOODS_DESCRIPTION_len_chars',
       'subtokenization_indicator'],
      dtype='object')

In [18]:
df

,HS06,GOODS_DESCRIPTION,HS04,HS02,full_eng,GOODS_DESCRIPTION_len_words,GOODS_DESCRIPTION_len_chars,subtokenization_indicator
0,271019,BRAKE FLUID DOT 4 50X200ML,2710,27,Petroleum oils and oils obtained from bitumino...,5,26,1.600000
1,847710,PLASTIC INJECTION MOULD MODEL 21A 110G DSM1010...,8477,84,Machinery for working rubber or plastics or fo...,7,53,2.428571
2,844399,LCD ASSEMBLY,8443,84,Printing machinery used for printing by means ...,2,12,1.000000
3,848280,BEARING 22238 KCAW33C3 BRAND MCB,8482,84,"Ball or roller bearings. && - Other, including...",5,32,2.200000
4,630900,USED HANDBAGS AND WALLETS,6309,63,NaN,4,25,1.500000
...,...,...,...,...,...,...,...,...
267775,854239,PCB OPTIONAL ADD. KROPT V4.0 (NEW OUT PUT CARD),8542,85,Electronic integrated circuits. && - Electroni...,9,47,2.000000
267776,842091,CYLINDER (SDA80*10F003000001A),8420,84,"Calendering or other rolling machines, other t...",2,30,7.500000
267777,830249,BEOTIC DEVICE,8302,83,"Base metal mountings, fittings and similar art...",2,13,1.500000
267778,901180,COMPOUND BINOCULAR MICROSCOPE,9011,90,"Compound optical microscopes, including those ...",3,29,1.666667


Stats evaluation

In [19]:
df.describe()

,GOODS_DESCRIPTION_len_words,GOODS_DESCRIPTION_len_chars,subtokenization_indicator
count,267780.000000,267780.000000,267780.000000
mean,4.588498,29.223411,1.947238
std,2.887145,18.470528,1.207913
min,1.000000,2.000000,1.000000
25%,3.000000,17.000000,1.250000
50%,4.000000,25.000000,1.666667
75%,6.000000,36.000000,2.285714
max,41.000000,150.000000,59.000000


Getting Stats from Grouped HS Codes

In [20]:
agg_col_for_sum = ['HS06']
                #    'HS04',
                #    'HS02']

agg_dict = {col: ['count'] for col in agg_col_for_sum}

agg_col_for_stats = ['GOODS_DESCRIPTION_len_words', 
                     'GOODS_DESCRIPTION_len_chars', 
                     'subtokenization_indicator']

agg_dict.update({col: ['sum','min', 'mean', 'median', 'max', 'std'] for col in agg_col_for_stats})


In [21]:
agg_dict

{'HS06': ['count'],
 'GOODS_DESCRIPTION_len_words': ['sum', 'min', 'mean', 'median', 'max', 'std'],
 'GOODS_DESCRIPTION_len_chars': ['sum', 'min', 'mean', 'median', 'max', 'std'],
 'subtokenization_indicator': ['sum', 'min', 'mean', 'median', 'max', 'std']}

HS06 expanded stats

In [22]:
df

,HS06,GOODS_DESCRIPTION,HS04,HS02,full_eng,GOODS_DESCRIPTION_len_words,GOODS_DESCRIPTION_len_chars,subtokenization_indicator
0,271019,BRAKE FLUID DOT 4 50X200ML,2710,27,Petroleum oils and oils obtained from bitumino...,5,26,1.600000
1,847710,PLASTIC INJECTION MOULD MODEL 21A 110G DSM1010...,8477,84,Machinery for working rubber or plastics or fo...,7,53,2.428571
2,844399,LCD ASSEMBLY,8443,84,Printing machinery used for printing by means ...,2,12,1.000000
3,848280,BEARING 22238 KCAW33C3 BRAND MCB,8482,84,"Ball or roller bearings. && - Other, including...",5,32,2.200000
4,630900,USED HANDBAGS AND WALLETS,6309,63,NaN,4,25,1.500000
...,...,...,...,...,...,...,...,...
267775,854239,PCB OPTIONAL ADD. KROPT V4.0 (NEW OUT PUT CARD),8542,85,Electronic integrated circuits. && - Electroni...,9,47,2.000000
267776,842091,CYLINDER (SDA80*10F003000001A),8420,84,"Calendering or other rolling machines, other t...",2,30,7.500000
267777,830249,BEOTIC DEVICE,8302,83,"Base metal mountings, fittings and similar art...",2,13,1.500000
267778,901180,COMPOUND BINOCULAR MICROSCOPE,9011,90,"Compound optical microscopes, including those ...",3,29,1.666667


In [23]:
df.loc[df['HS06'] == '010121']

,HS06,GOODS_DESCRIPTION,HS04,HS02,full_eng,GOODS_DESCRIPTION_len_words,GOODS_DESCRIPTION_len_chars,subtokenization_indicator
368,010121,LIVE FARM HORSE - FS STRADIVARIUS,0101,01,"Live horses, asses, mules and hinnies. && - Ho...",6,33,1.666667
99427,010121,LIVE BREEDING FARM HORSE SILOHA NASMAT,0101,01,"Live horses, asses, mules and hinnies. && - Ho...",6,38,1.500000
107885,010121,LIVE BREEDING FARM HORSE,0101,01,"Live horses, asses, mules and hinnies. && - Ho...",4,24,1.000000
216198,010121,LIVE BREEDING FARM HORSE LULUS PICASSOS VISUAL,0101,01,"Live horses, asses, mules and hinnies. && - Ho...",7,46,1.285714
244031,010121,LIVE FARM HORSE - SIDI AL JASIR,0101,01,"Live horses, asses, mules and hinnies. && - Ho...",7,31,1.285714


In [24]:
stats_hs06 = df.groupby('HS06').agg(agg_dict)

stats_hs06

HS06 GOODS_DESCRIPTION_len_words                                     \
       count                         sum min      mean median max       std   
HS06                                                                          
010121     5                          30   4  6.000000    6.0   7  1.224745   
010130     1                           2   2  2.000000    2.0   2       NaN   
010190     1                           3   3  3.000000    3.0   3       NaN   
010221     5                          20   2  4.000000    3.0   7  2.345208   
010229     2                           8   3  4.000000    4.0   5  1.414214   
...      ...                         ...  ..       ...    ...  ..       ...   
970190    26                          98   1  3.769231    2.5  13  3.037205   
970200     1                           3   3  3.000000    3.0   3       NaN   
970300    31                         100   1  3.225806    2.0  12  2.261411   
970400     6                          16   1  2.666667    2.5   4  1.211060   
970500     4                          11   1  2.750000    2.0   6  2.217356   

       GOODS_DESCRIPTION_len_chars                                       \
                               sum min       mean median max        std   
HS06                                                                      
010121                         172  24  34.400000   33.0  46   8.203658   
010130                          16  16  16.000000   16.0  16        NaN   
010190                          15  15  15.000000   15.0  15        NaN   
010221                         103   8  20.600000   19.0  34  11.631853   
010229                          47  22  23.500000   23.5  25   2.121320   
...                            ...  ..        ...    ...  ..        ...   
970190                         621   6  23.884615   18.0  71  17.673318   
970200                          14  14  14.000000   14.0  14        NaN   
970300                         672   9  21.677419   17.0  74  14.246373   
970400                         100   7  16.666667   14.0  30   9.025889   
970500                          75   8  18.750000   15.0  37  12.632630   

       subtokenization_indicator                                               
                             sum  min      mean    median       max       std  
HS06                                                                           
010121                  6.738095  1.0  1.347619  1.285714  1.666667  0.251751  
010130                  1.500000  1.5  1.500000  1.500000  1.500000       NaN  
010190                  1.000000  1.0  1.000000  1.000000  1.000000       NaN  
010221                  7.000000  1.0  1.400000  1.333333  2.000000  0.434613  
010229                  3.333333  1.0  1.666667  1.666667  2.333333  0.942809  
...                          ...  ...       ...       ...       ...       ...  
970190                 37.526496  1.0  1.443327  1.138889  4.000000  0.708957  
970200                  1.000000  1.0  1.000000  1.000000  1.000000       NaN  
970300                 53.633333  1.0  1.730108  1.500000  4.000000  0.844376  
970400                  7.000000  1.0  1.166667  1.000000  1.750000  0.302765  
970500                  6.666667  1.0  1.666667  1.500000  2.666667  0.816497  

[4190 rows x 19 columns]

In [25]:
print(stats_hs06.head(5).to_markdown(), "\n")

print(stats_hs06.tail(5).to_markdown(), "\n")


|   HS06 |   ('HS06', 'count') |   ('GOODS_DESCRIPTION_len_words', 'sum') |   ('GOODS_DESCRIPTION_len_words', 'min') |   ('GOODS_DESCRIPTION_len_words', 'mean') |   ('GOODS_DESCRIPTION_len_words', 'median') |   ('GOODS_DESCRIPTION_len_words', 'max') |   ('GOODS_DESCRIPTION_len_words', 'std') |   ('GOODS_DESCRIPTION_len_chars', 'sum') |   ('GOODS_DESCRIPTION_len_chars', 'min') |   ('GOODS_DESCRIPTION_len_chars', 'mean') |   ('GOODS_DESCRIPTION_len_chars', 'median') |   ('GOODS_DESCRIPTION_len_chars', 'max') |   ('GOODS_DESCRIPTION_len_chars', 'std') |   ('subtokenization_indicator', 'sum') |   ('subtokenization_indicator', 'min') |   ('subtokenization_indicator', 'mean') |   ('subtokenization_indicator', 'median') |   ('subtokenization_indicator', 'max') |   ('subtokenization_indicator', 'std') |
|-------:|--------------------:|-----------------------------------------:|-----------------------------------------:|------------------------------------------:|-------------------------------

HS04 expanded stats

In [26]:
stats_hs04 = df.groupby('HS04').agg(agg_dict)

stats_hs04

HS06 GOODS_DESCRIPTION_len_words                                     \
     count                         sum min      mean median max       std   
HS04                                                                        
0101     7                          35   2  5.000000    6.0   7  2.000000   
0102    10                          35   2  3.500000    3.0   7  1.840894   
0103     2                           3   1  1.500000    1.5   2  0.707107   
0104     3                           6   1  2.000000    1.0   4  1.732051   
0105    76                         591   2  7.776316    7.0  20  3.900765   
...    ...                         ...  ..       ...    ...  ..       ...   
9701    52                         166   1  3.192308    3.0  13  2.376671   
9702     1                           3   3  3.000000    3.0   3       NaN   
9703    31                         100   1  3.225806    2.0  12  2.261411   
9704     6                          16   1  2.666667    2.5   4  1.211060   
9705     4                          11   1  2.750000    2.0   6  2.217356   

     GOODS_DESCRIPTION_len_chars                                        \
                             sum min       mean median  max        std   
HS04                                                                     
0101                         203  15  29.000000   31.0   46  11.401754   
0102                         179   8  17.900000   15.5   34   9.757618   
0103                          20   4  10.000000   10.0   16   8.485281   
0104                          33   4  11.000000    9.0   20   8.185353   
0105                        3387  14  44.565789   37.0  124  23.405034   
...                          ...  ..        ...    ...  ...        ...   
9701                        1111   6  21.365385   17.0   71  14.585823   
9702                          14  14  14.000000   14.0   14        NaN   
9703                         672   9  21.677419   17.0   74  14.246373   
9704                         100   7  16.666667   14.0   30   9.025889   
9705                          75   8  18.750000   15.0   37  12.632630   

     subtokenization_indicator                                               
                           sum  min      mean    median       max       std  
HS04                                                                         
0101                  9.238095  1.0  1.319728  1.285714  1.666667  0.255646  
0102                 13.333333  1.0  1.333333  1.000000  2.333333  0.496904  
0103                  3.000000  1.0  1.500000  1.500000  2.000000  0.707107  
0104                  3.500000  1.0  1.166667  1.000000  1.500000  0.288675  
0105                107.874584  1.0  1.419402  1.285714  2.687500  0.411470  
...                        ...  ...       ...       ...       ...       ...  
9701                 71.359829  1.0  1.372304  1.000000  4.000000  0.643291  
9702                  1.000000  1.0  1.000000  1.000000  1.000000       NaN  
9703                 53.633333  1.0  1.730108  1.500000  4.000000  0.844376  
9704                  7.000000  1.0  1.166667  1.000000  1.750000  0.302765  
9705                  6.666667  1.0  1.666667  1.500000  2.666667  0.816497  

[1133 rows x 19 columns]

HS02 expanded stats

In [27]:
stats_hs02 = df.groupby('HS02').agg(agg_dict)

stats_hs02

HS06 GOODS_DESCRIPTION_len_words                                     \
     count                         sum min      mean median max       std   
HS02                                                                        
01     129                         775   1  6.007752    5.0  20  3.859842   
02     293                        1558   1  5.317406    5.0  14  3.047277   
03      62                         182   1  2.935484    2.0   9  1.608084   
04    1233                        6844   1  5.550689    5.0  20  3.574472   
05      15                          36   1  2.400000    2.0   5  1.055597   
...    ...                         ...  ..       ...    ...  ..       ...   
93      64                         242   1  3.781250    3.0  12  2.675395   
94    7921                       33169   1  4.187476    3.0  32  2.764096   
95    4191                       17044   1  4.066810    4.0  28  2.236884   
96    3091                       12455   1  4.029440    3.0  22  2.460847   
97      94                         296   1  3.148936    2.5  13  2.238267   

     GOODS_DESCRIPTION_len_chars                                        \
                             sum min       mean median  max        std   
HS02                                                                     
01                          4417   4  34.240310   30.0  124  22.967972   
02                          9857   4  33.641638   32.0   89  15.813486   
03                          1124   4  18.129032   14.5   54  10.857473   
04                         43568   4  35.334955   30.0   95  19.138776   
05                           205   5  13.666667   14.0   30   5.802298   
...                          ...  ..        ...    ...  ...        ...   
93                          1578   4  24.656250   20.0   68  16.408688   
94                        204300   3  25.792198   21.0  150  17.272204   
95                        100155   3  23.897638   21.0  150  12.980664   
96                         75242   3  24.342284   21.0  132  14.985277   
97                          1972   6  20.978723   17.0   74  13.922967   

     subtokenization_indicator                                           
                           sum  min      mean    median   max       std  
HS02                                                                     
01                  181.771013  1.0  1.409078  1.285714   3.0  0.451324  
02                  846.670169  1.0  2.889659  1.666667  33.0  4.756294  
03                   96.351587  1.0  1.554058  1.416667   3.4  0.608181  
04                 3559.741407  1.0  2.887057  1.750000  36.0  4.596942  
05                   20.883333  1.0  1.392222  1.333333   2.0  0.401429  
...                        ...  ...       ...       ...   ...       ...  
93                  130.598810  1.0  2.040606  1.750000   6.5  1.173693  
94                12475.902712  1.0  1.575041  1.333333  16.0  0.859632  
95                 6111.075169  1.0  1.458142  1.333333  10.5  0.631161  
96                 5138.436948  1.0  1.662387  1.500000  10.5  0.816961  
97                  139.659829  1.0  1.485743  1.055556   4.0  0.721784  

[96 rows x 19 columns]

Saving stats

In [28]:
# stats_hs06.to_csv('eda_stats_hs06.csv')
# stats_hs04.to_csv('eda_stats_hs04.csv')
# stats_hs02.to_csv('eda_stats_hs02.csv')

Reports to html format

In [29]:
from ydata_profiling import ProfileReport

# # flatening columns names
# stats_hs06.columns = ['_'.join(col) for col in stats_hs06.columns]
# stats_hs04.columns = ['_'.join(col) for col in stats_hs04.columns]
# stats_hs02.columns = ['_'.join(col) for col in stats_hs02.columns]

# ProfileReport(stats_hs06, title="HS06 Profiling Report").to_file("eda_stats_hs06.html")
# ProfileReport(stats_hs04, title="HS04 Profiling Report").to_file("eda_stats_hs04.html")
# ProfileReport(stats_hs02, title="HS02 Profiling Report").to_file("eda_stats_hs02.html")


### Embeddings with DistilBERT

Using pretrained NLP model for contextual representation

distilbert-base-uncased

In [30]:
# Custom Dataset for embeddings
class HSDescEmbedDataset(Dataset):
    def __init__(self, dataframe, tokenizer, desc_col='', label_col='', max_length=500):
        self.texts = dataframe[desc_col].fillna("").tolist()
        self.labels = dataframe[label_col].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': self.labels[idx]
        }

# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
model.to(device)

# Function to extract [CLS] embeddings
def get_embeddings(dataloader, model, device):
    all_embeds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attn = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attn)
            # DistilBERT does not have pooler; use first token hidden state
            cls_embeds = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeds.append(cls_embeds)
            all_labels.extend(batch['label'])
    return np.vstack(all_embeds), np.array(all_labels)

In [31]:
torch.cuda.is_available()

True

#### Goods description emb

In [32]:
df_emb = df.sample(n=len(df)//20)

In [33]:
df_emb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13389 entries, 186910 to 136847
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   HS06                         13389 non-null  object 
 1   GOODS_DESCRIPTION            13389 non-null  object 
 2   HS04                         13389 non-null  object 
 3   HS02                         13389 non-null  object 
 4   full_eng                     12754 non-null  object 
 5   GOODS_DESCRIPTION_len_words  13389 non-null  int64  
 6   GOODS_DESCRIPTION_len_chars  13389 non-null  int64  
 7   subtokenization_indicator    13389 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 941.4+ KB


Generating embs from goods descriptions

In [34]:
# # Build dataset and loader
# ds = HSDescEmbedDataset(df_emb, tokenizer, desc_col='GOODS_DESCRIPTION', label_col='HS02', max_length=500)
# loader = DataLoader(ds, batch_size=32, shuffle=False)

# embeds, labels = get_embeddings(loader, model, device)
# print("Embeddings shape:", embeds.shape)

Saving embeddings

In [35]:
# EMB_n_LABELS_FILE = "good_desc_sample_embs_labels.npz"

# np.savez(EMB_n_LABELS_FILE, embeds=embeds, labels=labels)
# print("💾 Saved embeddings and labels to disk.")

Loading embeddings

In [36]:
EMB_n_LABELS_FILE = "good_desc_sample_embs_labels.npz"

if os.path.exists(EMB_n_LABELS_FILE):
    data = np.load(EMB_n_LABELS_FILE, allow_pickle=True)
    embeds = data["embeds"]
    labels = data["labels"]
    print("✅ Loaded cached embeddings and labels")
else:
    raise FileNotFoundError(f"{EMB_n_LABELS_FILE} not found – run get_embeddings first.")

✅ Loaded cached embeddings and labels


Viz embeddings with PCA

In [37]:
def pca_viz_embs(embeds, labels, n_componets=2, title='',
                 legend_title='', html_output_file=''):

    pca = PCA(n_components=n_componets)
    emb = pca.fit_transform(embeds)

    fig = go.Figure()
    for chap in np.unique(labels):
        mask = labels == chap
        if n_componets==2:
            fig.add_trace(go.Scatter(
                x=emb[mask, 0],
                y=emb[mask, 1],
                mode='markers',
                name=chap,
                marker=dict(size=4),
                hovertext=[chap]*mask.sum(),
                hoverinfo='text'
            ))
        elif n_componets==3:
            fig.add_trace(go.Scatter3d(
                x=emb[mask, 0],
                y=emb[mask, 1],
                z=emb[mask, 2],
                mode='markers',
                name=chap,
                marker=dict(size=3),
                hovertext=[chap]*mask.sum(),
                hoverinfo='text'
            ))
        else:
            return 
    if n_componets==2:
        scene = dict(
            xaxis_title="PC1", yaxis_title="PC2")
    elif n_componets==3:
        scene = dict(
            xaxis_title="PC1", yaxis_title="PC2", zaxis_title="PC3")
    else:
        return 
    fig.update_layout(
        title=title,
        scene=scene,
        legend_title=legend_title,
        width=1200, height=900
    )
    fig.write_html(html_output_file)
    print(f"📄 Saved plot to {html_output_file}")

In [38]:
# pca_viz_embs(embeds, labels, n_componets=2,
#              title="Goods Description sampled - 2D PCA of DistilBERT Embeddings",
#              legend_title="HS02 Chapter",
#              html_output_file="embeddings_2d_goods_desc_sampled.html")


In [39]:
# pca_viz_embs(embeds, labels, n_componets=3,
#              title="Goods Description sampled - 3D PCA of DistilBERT Embeddings",
#              legend_title="HS02 Chapter",
#              html_output_file="embeddings_3d_goods_desc_sampled.html")

#### HS06 nomenclature emb 

In [40]:
# Add HS02 (chapter) and HS04 (heading)
df_hs06['HS04'] = df_hs06.index.str[:4]
df_hs06['HS02'] = df_hs06.index.str[:2]

df_hs06

,full_eng,HS04,HS02
hs06,,,
010120,"Live horses, asses, mules and hinnies. && - Ho...",0101,01
010121,"Live horses, asses, mules and hinnies. && - Ho...",0101,01
010129,"Live horses, asses, mules and hinnies. && - Ho...",0101,01
010130,"Live horses, asses, mules and hinnies. && - Asses",0101,01
010190,"Live horses, asses, mules and hinnies. && - Other",0101,01
...,...,...,...
961590,"Combs, hair-slides and the like; hairpins, cur...",9615,96
961610,"Scent sprays and similar toilet sprays, and mo...",9616,96
961620,"Scent sprays and similar toilet sprays, and mo...",9616,96


Generating embs from full_eng nomenclature

In [41]:
# # Build dataset and loader
# ds = HSDescEmbedDataset(df_hs06, tokenizer, desc_col='full_eng', label_col='HS02', max_length=500)
# loader = DataLoader(ds, batch_size=32, shuffle=False)

# embeds, labels = get_embeddings(loader, model, device)
# print("Embeddings shape:", embeds.shape)

Saving embeddings

In [42]:
# EMB_n_LABELS_FILE = "hs02_fulleng_embs_labels.npz"

# np.savez(EMB_n_LABELS_FILE, embeds=embeds, labels=labels)
# print("💾 Saved embeddings and labels to disk.")

Loading embeddings

In [43]:
EMB_n_LABELS_FILE = "hs02_fulleng_embs_labels.npz"

if os.path.exists(EMB_n_LABELS_FILE):
    data = np.load(EMB_n_LABELS_FILE, allow_pickle=True)
    embeds = data["embeds"]
    labels = data["labels"]
    print("✅ Loaded cached embeddings and labels")
else:
    raise FileNotFoundError(f"{EMB_n_LABELS_FILE} not found – run get_embeddings first.")

✅ Loaded cached embeddings and labels


Viz embeddings with PCA

In [44]:
# pca_viz_embs(embeds, labels, n_componets=2,
#              title="HS06 full eng - 2D PCA of DistilBERT Embeddings",
#              legend_title="HS02 Chapter",
#              html_output_file="embeddings_2d_hs06_full_eng.html")

In [45]:
# pca_viz_embs(embeds, labels, n_componets=3,
#              title="HS06 full eng - 3D PCA of DistilBERT Embeddings",
#              legend_title="HS02 Chapter",
#              html_output_file="embeddings_3d_hs06_full_eng.html")

#### Goods desc vs. HS06 full eng embs

For similarity analysis

Full goods descriptions embs

In [46]:
# # Goods description (gd) for HS06 embeddings
# ds_gd = HSDescEmbedDataset(df, tokenizer, desc_col='GOODS_DESCRIPTION', label_col='HS06', max_length=500)
# loader_gd = DataLoader(ds_gd, batch_size=32, shuffle=False)

# gd_embeds, hs06_gd_labels = get_embeddings(loader_gd, model, device)
# print("Embeddings shape:", gd_embeds.shape)

Saving embs

In [47]:
# EMB_n_LABELS_FILE = "good_desc_embs_labels.npz"

# np.savez(EMB_n_LABELS_FILE, embeds=gd_embeds, labels=hs06_gd_labels)
# print("💾 Saved embeddings and labels to disk.")

HS06 full eng legal text embs

In [48]:
# # HS full eng (hs) sampled for HS06 embeddings
# ds_hs = HSDescEmbedDataset(df_hs06.reset_index(inplace=False), tokenizer, desc_col='full_eng', label_col='hs06', max_length=500)
# loader_hs = DataLoader(ds_hs, batch_size=32, shuffle=False)

# hs_embeds, hs_labels = get_embeddings(loader_hs, model, device)
# print("Embeddings shape:", hs_embeds.shape)

Saving embs

In [49]:
# EMB_n_LABELS_FILE = "hs06_fulleng_embs_labels.npz"

# np.savez(EMB_n_LABELS_FILE, embeds=hs_embeds, labels=hs_labels)
# print("💾 Saved embeddings and labels to disk.")

Caculating cosine similarity

In [50]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # Map HS06 code → corresponding legal text embedding
# # Normalize hs06 codes as string if necessary
# hs06_to_hsvec = {}
# for hs_code, emb in zip(hs_labels, hs_embeds):
#     hs06_to_hsvec[str(hs_code)] = emb

# # Compute cosine similarity for each GOODS_DESCRIPTION embedding
# cosine_sims = []

# for i in range(len(gd_embeds)):
#     hs_code = str(hs06_gd_labels[i])  # ensure string matching
#     gd_vec = gd_embeds[i].reshape(1, -1)  # shape (1, dim)

#     hs_vec = hs06_to_hsvec.get(hs_code)

#     if hs_vec is not None:
#         hs_vec = hs_vec.reshape(1, -1)
#         sim = cosine_similarity(gd_vec, hs_vec)[0][0]
#     else:
#         sim = np.nan  # if no match found, assign NaN

#     cosine_sims.append(sim)

# # Add results to df_emb
# df['cosine_sim_gd_vs_hs_text'] = cosine_sims

In [51]:
df.describe()

,GOODS_DESCRIPTION_len_words,GOODS_DESCRIPTION_len_chars,subtokenization_indicator
count,267780.000000,267780.000000,267780.000000
mean,4.588498,29.223411,1.947238
std,2.887145,18.470528,1.207913
min,1.000000,2.000000,1.000000
25%,3.000000,17.000000,1.250000
50%,4.000000,25.000000,1.666667
75%,6.000000,36.000000,2.285714
max,41.000000,150.000000,59.000000


Saving dataframe with all features and with cosine similarity

In [52]:
# df.to_csv("data/df_all_eda_features.csv")

In [53]:
df = pd.read_csv("data/df_all_eda_features.csv", index_col=0)

Top similarities

In [54]:
cols = ['HS06', 'GOODS_DESCRIPTION', 'full_eng', 'cosine_sim_gd_vs_hs_text']

In [55]:
print("Top 10 cosine similarity between goods description and HS06 full eng description:\n")
print(df[cols].dropna().sort_values(by='cosine_sim_gd_vs_hs_text', ascending=False).head(10).to_markdown(), "\n")

Top 10 cosine similarity between goods description and HS06 full eng description:

|        |   HS06 | GOODS_DESCRIPTION                                                                | full_eng                                                                                                                           |   cosine_sim_gd_vs_hs_text |
|-------:|-------:|:---------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|---------------------------:|
|  45158 | 640299 | Other:Other footwear with outer soles and uppers of rubber or pla:Other footwear | Other footwear with outer soles and uppers of rubber or plastics. && - Other footwear : && -- Other                                |                   0.98534  |
| 189750 | 520939 | Other fabrics:Woven fabrics of cotton, containing 85 % or more by weight of:Dyed | Woven fabrics of co

Top distances

In [56]:
print("Bottom 10 cosine similarity between goods description and HS06 full eng description:\n")
print(df[cols].dropna().sort_values(by='cosine_sim_gd_vs_hs_text', ascending=False).tail(10).to_markdown(), "\n")

Bottom 10 cosine similarity between goods description and HS06 full eng description:

|        |   HS06 | GOODS_DESCRIPTION                         | full_eng                                                                                                                                                                                                                               |   cosine_sim_gd_vs_hs_text |
|-------:|-------:|:------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------:|
| 247611 | 741820 | SHOWER HEAD SQUARE BLACK 260X 190mm       | Table, kitchen or other household articles and parts thereof, of copper; pot scourers and scouring or polishing pads, gloves and the like, of copper; sanitary ware and parts thereof, of copper. && -

Average similarity

In [57]:
sim_mean = df['cosine_sim_gd_vs_hs_text'].mean()
upper = sim_mean + sim_mean / 100
lower = sim_mean - sim_mean / 100

In [58]:
print("Regular cosine similarity between goods description and HS06 full eng description:\n")
print(df[cols].loc[(df['cosine_sim_gd_vs_hs_text'] < upper) & (df['cosine_sim_gd_vs_hs_text'] > lower)].head(10), "\n")

Regular cosine similarity between goods description and HS06 full eng description:

      HS06                               GOODS_DESCRIPTION  \
5   853950                    ASSY LED Base Strobe Upgrade   
9   220870                  VODKA FRAISE JELZIN STRAWBERRY   
18  620590                       SHORT SLEEVE REPAIR SHIRT   
27  732620                                      HANGER ROD   
32  843143             8-3/8SH Extension Overshot C-17208   
41  870323                              SUZUKI ESCUDO 2006   
48  841899  Spare Parts for 10 TR Air Cooled Water Chiller   
57  871120              USED CHANGZHOU KWANGYANG MOTORBYKE   
61  940540                          SURFACE MOUNTED LIGHTS   
67  842131                        FILTER ASSY, AIR CLEANER   

                                             full_eng  \
5   Electric filament or discharge lamps, includin...   
9   Undenatured ethyl alcohol of an alcoholic stre...   
18  Men's or boys' shirts. && - Of other textile m...   
27  O

Getting new stats from grouped HS06 codes

In [59]:
agg_col_for_sum = ['HS06']

agg_dict = {col: ['count'] for col in agg_col_for_sum}

agg_col_for_stats = ['GOODS_DESCRIPTION_len_words', 
                     'GOODS_DESCRIPTION_len_chars', 
                     'subtokenization_indicator',
                     'cosine_sim_gd_vs_hs_text']

agg_dict.update({col: ['sum','min', 'mean', 'median', 'max', 'std'] for col in agg_col_for_stats})


sum of cosine_sim_gd_vs_hs_text is not relevant

In [60]:
agg_dict

{'HS06': ['count'],
 'GOODS_DESCRIPTION_len_words': ['sum', 'min', 'mean', 'median', 'max', 'std'],
 'GOODS_DESCRIPTION_len_chars': ['sum', 'min', 'mean', 'median', 'max', 'std'],
 'subtokenization_indicator': ['sum', 'min', 'mean', 'median', 'max', 'std'],
 'cosine_sim_gd_vs_hs_text': ['sum', 'min', 'mean', 'median', 'max', 'std']}

In [61]:
stats_hs06 = df.groupby('HS06').agg(agg_dict)

stats_hs06

HS06 GOODS_DESCRIPTION_len_words                                     \
       count                         sum min      mean median max       std   
HS06                                                                          
10121      5                          30   4  6.000000    6.0   7  1.224745   
10130      1                           2   2  2.000000    2.0   2       NaN   
10190      1                           3   3  3.000000    3.0   3       NaN   
10221      5                          20   2  4.000000    3.0   7  2.345208   
10229      2                           8   3  4.000000    4.0   5  1.414214   
...      ...                         ...  ..       ...    ...  ..       ...   
970190    26                          98   1  3.769231    2.5  13  3.037205   
970200     1                           3   3  3.000000    3.0   3       NaN   
970300    31                         100   1  3.225806    2.0  12  2.261411   
970400     6                          16   1  2.666667    2.5   4  1.211060   
970500     4                          11   1  2.750000    2.0   6  2.217356   

       GOODS_DESCRIPTION_len_chars                 ...  \
                               sum min       mean  ...   
HS06                                               ...   
10121                          172  24  34.400000  ...   
10130                           16  16  16.000000  ...   
10190                           15  15  15.000000  ...   
10221                          103   8  20.600000  ...   
10229                           47  22  23.500000  ...   
...                            ...  ..        ...  ...   
970190                         621   6  23.884615  ...   
970200                          14  14  14.000000  ...   
970300                         672   9  21.677419  ...   
970400                         100   7  16.666667  ...   
970500                          75   8  18.750000  ...   

       subtokenization_indicator                                \
                            mean    median       max       std   
HS06                                                             
10121                   1.347619  1.285714  1.666667  0.251751   
10130                   1.500000  1.500000  1.500000       NaN   
10190                   1.000000  1.000000  1.000000       NaN   
10221                   1.400000  1.333333  2.000000  0.434613   
10229                   1.666667  1.666667  2.333333  0.942809   
...                          ...       ...       ...       ...   
970190                  1.443327  1.138889  4.000000  0.708957   
970200                  1.000000  1.000000  1.000000       NaN   
970300                  1.730108  1.500000  4.000000  0.844376   
970400                  1.166667  1.000000  1.750000  0.302765   
970500                  1.666667  1.500000  2.666667  0.816497   

       cosine_sim_gd_vs_hs_text                                          \
                            sum       min      mean    median       max   
HS06                                                                      
10121                  4.493902  0.889913  0.898780  0.894978  0.912786   
10130                  0.822331  0.822331  0.822331  0.822331  0.822331   
10190                  0.866746  0.866746  0.866746  0.866746  0.866746   
10221                  4.574999  0.876037  0.915000  0.915777  0.937644   
10229                  1.812984  0.888580  0.906492  0.906492  0.924403   
...                         ...       ...       ...       ...       ...   
970190                21.213793  0.742387  0.815915  0.813457  0.902661   
970200                 0.000000       NaN       NaN       NaN       NaN   
970300                 0.000000       NaN       NaN       NaN       NaN   
970400                 0.000000       NaN       NaN       NaN       NaN   
970500                 0.000000       NaN       NaN       NaN       NaN   

                  
             std  
HS06              
10121   0.009314  
10130        NaN  
10190        NaN 

Saving new stats

In [62]:
stats_hs06.to_csv('eda_stats_hs06_with_similarity.csv')

Report to html format

In [63]:
stats_hs06.columns = ['_'.join(col) for col in stats_hs06.columns]

ProfileReport(stats_hs06, title="HS06 Goods Description EDA").to_file("eda_stats_hs06_with_similarity.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

EDA Finished

Write report and present

______